In [264]:
import pandas as pd
import numpy as np

### EDA (Faults)

In [102]:
faults = pd.read_csv("../data/J1939Faults.csv", nrows = 100)

In [104]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [208]:
faults.shape

(100, 20)

In [106]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RecordID               100 non-null    int64  
 1   ESS_Id                 100 non-null    int64  
 2   EventTimeStamp         100 non-null    object 
 3   eventDescription       82 non-null     object 
 4   actionDescription      0 non-null      float64
 5   ecuSoftwareVersion     100 non-null    object 
 6   ecuSerialNumber        100 non-null    object 
 7   ecuModel               100 non-null    object 
 8   ecuMake                100 non-null    object 
 9   ecuSource              100 non-null    int64  
 10  spn                    100 non-null    int64  
 11  fmi                    100 non-null    int64  
 12  active                 100 non-null    bool   
 13  activeTransitionCount  100 non-null    int64  
 14  faultValue             0 non-null      float64
 15  Equipme

In [108]:
faults.value_counts(normalize= True)

Series([], Name: proportion, dtype: float64)

In [110]:
faults.isnull().all() # checking to see which columns are completely NaN values
                      # And there are two: actionDescription, faultValue  - i'll drop these

RecordID                 False
ESS_Id                   False
EventTimeStamp           False
eventDescription         False
actionDescription         True
ecuSoftwareVersion       False
ecuSerialNumber          False
ecuModel                 False
ecuMake                  False
ecuSource                False
spn                      False
fmi                      False
active                   False
activeTransitionCount    False
faultValue                True
EquipmentID              False
MCTNumber                False
Latitude                 False
Longitude                False
LocationTimeStamp        False
dtype: bool

In [254]:
faults.isnull().all(axis=1).value_counts() # checking to see if there are any rows that are completely NaN values
                                           # And there aren't any

False    100
Name: count, dtype: int64

In [114]:
faults.loc[0]

RecordID                                                       1
ESS_Id                                                    990349
EventTimeStamp                           2015-02-21 10:47:13.000
eventDescription         Low (Severity Low) Engine Coolant Level
actionDescription                                            NaN
ecuSoftwareVersion                                       unknown
ecuSerialNumber                                          unknown
ecuModel                                                 unknown
ecuMake                                                  unknown
ecuSource                                                      0
spn                                                          111
fmi                                                           17
active                                                      True
activeTransitionCount                                          2
faultValue                                                   NaN
EquipmentID              

Looking at the first record, here is a breakdown of the important values.

* ESS_Id, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, faultValue, and MCTNumber are unlikely to provide any predictive value.
* We can see the time of the event in the **EventTimeStamp** column. Note that this may be different from the **LocationTimeStamp** value, which indicates when the Latitude/Longitude values were recorded.
* The **spn** and **fmi** columns together indicate the type of fault, and there may be a description of that fault in the **eventDescription** column, although this column is sometimes missing.
* Faults are recorded when the light goes on and when it goes off, which is indicated by the **active** column, with True indicating the light turning on and False indicating turning off. The number of times the code has been set or unset is in the **faultValue** column, although this value can be unreliable. 
* Each truck has an identifier, the **EquipmentID** value.
* Each record can be linked to the on-board diagnostics data through the **RecordID** column.

In [116]:
print(faults.duplicated().value_counts(normalize= True))
print(faults[faults.duplicated()]) 
# I am trying to identify duplicated rows and there are none, so I don't need to drop any. 

False    1.0
Name: proportion, dtype: float64
Empty DataFrame
Columns: [RecordID, ESS_Id, EventTimeStamp, eventDescription, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, spn, fmi, active, activeTransitionCount, faultValue, EquipmentID, MCTNumber, Latitude, Longitude, LocationTimeStamp]
Index: []


In [ ]:
print(faults['ESS_Id'].isnull().value_counts(normalize = True))
print(faults['actionDescription'].isnull().value_counts(normalize = True))
print(faults['ecuSoftwareVersion'].isnull().value_counts(normalize = True))
print(faults['ecuSerialNumber'].isnull().value_counts(normalize = True))
print(faults['ecuModel'].isnull().value_counts(normalize = True))
print(faults['ecuMake'].isnull().value_counts(normalize = True))
print(faults['ecuSource'].isnull().value_counts(normalize = True))
print(faults['faultValue'].isnull().value_counts(normalize = True))
print(faults['MCTNumber'].isnull().value_counts(normalize = True))

### Data Prepping (Faults)

In [210]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')
faults_prepped.shape
# dropped columns that contained only NaN values # dropped rows that contained only NaN values

(100, 18)

### EDA (Diagnostics)

In [ ]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv", nrows = 100)

In [27]:
diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [212]:
diagnostics.shape

(100, 4)

In [222]:
diagnostics['Name'].value_counts().count()

23

In [33]:
faults['RecordID'].value_counts(normalize=True)

RecordID
1      0.01
64     0.01
74     0.01
73     0.01
72     0.01
       ... 
31     0.01
30     0.01
29     0.01
28     0.01
100    0.01
Name: proportion, Length: 100, dtype: float64

In [25]:
diagnostics['FaultId'].value_counts()

FaultId
1     21
6     20
7     20
11    19
15     6
2      2
4      2
8      2
9      2
3      1
5      1
10     1
12     1
13     1
14     1
Name: count, dtype: int64

In [11]:
diagnostics.info()
# there are no NaN columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       100 non-null    int64 
 1   Name     100 non-null    object
 2   Value    100 non-null    object
 3   FaultId  100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.3+ KB


To get the on-board diagnostics at the time of the fault code, we can match the **RecordID** to the **FaultId**.

In [13]:
print(diagnostics.duplicated().value_counts(normalize= True))
print(diagnostics[diagnostics.duplicated()]) 
# there are no duplicate values in diagnostics

False    1.0
Name: proportion, dtype: float64
Empty DataFrame
Columns: [Id, Name, Value, FaultId]
Index: []


In [40]:
diagnostics.head(3)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1


### Data Prepping (Not-necessary)

In [ ]:
### Faults & Diagnostic Data Prepping

In [224]:
diagnostics_pivoted = diagnostics.pivot(index = 'FaultId', columns='Name', values='Value').reset_index()
diagnostics_pivoted

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,0,NaN,False,78.8,1023,True,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN
5,6,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,...,4.583399,NaN,True,111.2,1023,NaN,13.6022,3276.75,NaN,6.67
6,7,82.8,14.2825,False,64.6226,278736.7,188.6,80,39.44,210.0313,...,14.29175,NaN,True,78.8,1023,NaN,41.53478,3276.75,NaN,20.59
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN


In [234]:
faults_diagnostics_merged = pd.merge(faults_prepped, diagnostics_pivoted, left_on='RecordID', right_on='FaultId', how='outer')
faults_diagnostics_merged

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,0,NaN,False,78.8,1023,True,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,...,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,991581,2015-02-21 12:49:35.000,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,97,991600,2015-02-21 12:50:23.000,NaN,unknown,unknown,unknown,unknown,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,98,991601,2015-02-21 12:51:12.000,Incorrect Data Brake Signal Sensor 1,unknown,unknown,unknown,unknown,11,1067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,99,991602,2015-02-21 12:51:12.000,Incorrect Data Brake Signal Sensor 2,unknown,unknown,unknown,unknown,11,1068,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [248]:
print(f"There are this many duplicated values:", faults_diagnostics_merged.duplicated().value_counts()) # there are no duplicated rows. tbe.
faults_diagnostics_merged.isnull().all() # there are no completely NaN columns.

There are this many duplicated values: False    100
Name: count, dtype: int64


RecordID                     False
ESS_Id                       False
EventTimeStamp               False
eventDescription             False
ecuSoftwareVersion           False
ecuSerialNumber              False
ecuModel                     False
ecuMake                      False
ecuSource                    False
spn                          False
fmi                          False
active                       False
activeTransitionCount        False
EquipmentID                  False
MCTNumber                    False
Latitude                     False
Longitude                    False
LocationTimeStamp            False
FaultId                      False
AcceleratorPedal             False
BarometricPressure           False
CruiseControlActive          False
CruiseControlSetSpeed        False
DistanceLtd                  False
EngineCoolantTemperature     False
EngineLoad                   False
EngineOilPressure            False
EngineOilTemperature         False
EngineRpm           

In [258]:
faults_diagnostics_merged.isnull().all(axis=1).value_counts() # there are no completely empty rows. 

False    100
Name: count, dtype: int64

In [260]:
faults_diagnostics_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   RecordID                   100 non-null    int64  
 1   ESS_Id                     100 non-null    int64  
 2   EventTimeStamp             100 non-null    object 
 3   eventDescription           82 non-null     object 
 4   ecuSoftwareVersion         100 non-null    object 
 5   ecuSerialNumber            100 non-null    object 
 6   ecuModel                   100 non-null    object 
 7   ecuMake                    100 non-null    object 
 8   ecuSource                  100 non-null    int64  
 9   spn                        100 non-null    int64  
 10  fmi                        100 non-null    int64  
 11  active                     100 non-null    bool   
 12  activeTransitionCount      100 non-null    int64  
 13  EquipmentID                100 non-null    int64  


In [308]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km - roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle


threshold = 5  # Distance in km - this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_diagnostics_merged['Latitude'], faults_diagnostics_merged['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).

# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_diagnostics_merged = faults_diagnostics_merged[outside_vicinity]

faults_diagnostics_merged

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,0,NaN,False,78.8,1023,True,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,unknown,unknown,unknown,unknown,11,629,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,...,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,991551,2015-02-21 12:48:22.000,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,96,991581,2015-02-21 12:49:35.000,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,98,991601,2015-02-21 12:51:12.000,Incorrect Data Brake Signal Sensor 1,unknown,unknown,unknown,unknown,11,1067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,99,991602,2015-02-21 12:51:12.000,Incorrect Data Brake Signal Sensor 2,unknown,unknown,unknown,unknown,11,1068,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
distances.dtype

dtype('float64')

In [27]:
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
sfc.head()

/home/michael/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


For a large number of fault codes, there are multiple records. For example, if we look at the rows for the first fault in our dataset, we see that there are two rows.

In [28]:
(
    sfc
    .loc[sfc['SPN'] == 111]
    .loc[sfc['J1939 FMI'] == 17]
)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
1589,Y,2448,167,111,Not Mapped,Not Mapped,1,111,17,P2560,Maintenance,Maintenance,Coolant Level - Data Valid But Below Normal Op...,Low engine coolant level detected.
3540,Y,5167,167,Not Mapped,Not Mapped,Not Mapped,1,111,17,Not Mapped,Amber,Warning,Coolant Level - Data Valid But Below Normal Op...,NaN


Or even more.

In [29]:
(
    sfc
    .loc[sfc['SPN'] == 629]
    .loc[sfc['J1939 FMI'] == 12]
)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
180,Y,343,167,Not Mapped,254,0,12,629,12,P0607,Amber,Warning,Engine Control Module Warning Internal Hardwar...,ECM power supply errors have been detected.
689,Y,1116,167,Not Mapped,254,0,12,629,12,Not Mapped,Amber,Warning,Engine Control Module Critical Internal Failur...,ECM Internal failure has occurred.
854,Y,1388,167,Not Mapped,254,0,12,629,12,Not Mapped,NaN,NaN,Engine Control Module Data Lost - Bad Intellig...,The ECM data has been lost.
1019,Y,1597,167,Not Mapped,254,0,12,629,12,Not Mapped,Maintenance,Maintenance,Engine Control Module Critical Internal Failur...,The ECM has occurred an internal failure.
